# WHEN I"M DONE, I want to review what else I still need to calculate
# BASICALLY document the math for all metrics in overleaf and add to the latex wha I still need to calculate
# align the headers for this notebook and the category names in latex

In [1]:
import os
os.chdir('../')

In [59]:
import pandas as pd
from pathlib import Path
import numpy as np
import sys
import glob
import warnings
import random
from glob import glob 
import datetime
import itertools
import time
from multiprocessing import pool
from source.lib.helpers import *
from pandarallel import pandarallel
import ast
import swifter
from functools import reduce
import yaml
import re

In [3]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pandarallel.initialize(progress_bar = True)

indir_data = Path('drive/output/derived/contributor_stats/contributor_data')
outdir_data = Path('drive/output/derived/project_outcomes')

time_period = 6#int(sys.argv[1])
df_contributor_panel = pd.read_parquet(indir_data / f"major_contributors_major_months{time_period}_window732D_samplefull.parquet")

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Organizational Characteristics

**Libraries.io API**: Use the "Repos" query, query for "created at" date. 

<span style="color:red">Not done</span>

**Forks + Stars**: Obtained from GitHub Archive, calculates number of stars/forks per time-period  
<span style="color:blue">do validation on how accurate the statistics are</span>


In [4]:
def AggregateStarsForks(indir, time_period, colname):
    df_agg = pd.concat(
        (pd.read_csv(file, usecols = ['created_at','repo_name', 'org_login']) for file in indir.glob("*.csv")),
        ignore_index=True
    )
    df_agg = ImputeTimePeriod(df_agg, time_period)
    df = df_agg.groupby(['repo_name','time_period']).size().reset_index()
    df.rename(columns = {0:colname}, inplace = True)

    org_list = df_agg['org_login'].dropna().unique().tolist()
    return org_list, df

In [5]:
indir_watch = Path("drive/output/scrape/extract_github_data/watch_data")
indir_fork = Path("drive/output/scrape/extract_github_data/fork_data")
star_orgs, df_stars = AggregateStarsForks(indir_watch, time_period, 'stars')
df_stars.rename(columns = {'stars':'stars_accumulated'}, inplace = True)
fork_orgs, df_fork = AggregateStarsForks(indir_fork, time_period, 'forks')
df_fork.rename(columns = {'forks':'forks_accumulated'}, inplace = True)
initial_org_list = list(set(star_orgs + fork_orgs))

**Creator Type (Org/Indiv)**: Whether a project was created by an organization or an individual. I obtained this by obtaining all organization names from fork, star, issues, issue comments and PR, PR reviews and PR review comment data.

In [8]:
def GetUniqueOrgs(dir_path):
    dfs = [pd.read_csv(f, usecols=["org_login"], dtype=str) for f in dir_path.glob("*.csv")]
    if not dfs:
        return set()
    combined = pd.concat(dfs, ignore_index=True)
    return combined["org_login"].dropna().unique().tolist()

In [9]:
indir_issue_raw = Path("drive/output/scrape/extract_github_data/issue_data")
indir_issue_comment_raw = Path("drive/output/scrape/extract_github_data/issue_comment_data")
indir_pull_request_raw = Path("drive/output/scrape/extract_github_data/pull_request_data")
indir_pull_request_review_comment_raw = Path("drive/output/scrape/extract_github_data/pull_request_review_comment_data")
indir_pull_request_review_raw = Path("drive/output/scrape/extract_github_data/pull_request_review_data")

all_orgs = initial_org_list
for indir in [indir_issue_raw,indir_issue_comment_raw,indir_pull_request_raw, indir_pull_request_review_comment_raw,indir_pull_request_review_raw]:
    all_orgs += GetUniqueOrgs(indir)

unique_orgs_list = list(set(all_orgs))
print("Have collected {} unique organizations".format(len(unique_orgs_list)))

Have collected 2221 unique organizations


**Project License**: Used license obtained from PyPI as of scraping date. Two cleaning changes made:
- Changed all licenses with GPL to "GNU General Public License"
- Labelled all licenses that are not MIT/BSD/Apache/GNU as "other"

<span style="color:blue">Adjust for changes to license by projects, originally downloaded on October 27, 2023 or September 29, 2024</span>


In [157]:
indir_license = Path('output/derived/collect_github_repos')
def ProcessLicenseData(indir_license):
    special_licenses = ['MIT License', 'BSD License', 'Apache Software License', 'GNU General Public License']
    df = pd.read_csv(indir_license / 'linked_pypi_github.csv', index_col=0)
    df = df[['github repository', 'license']].query('`github repository` != "Unavailable"')
    df['license'] = df['license'].parallel_apply(ast.literal_eval).apply(
        lambda x: sorted(
            set(
                ['GNU General Public License' if 'GPL' in license else license if license in special_licenses else 'Other License' 
                 for license in x if license != "Unavailable"]
            )
        )
    )
    return df
def AddIndicators(df_license):
    df = df_license.rename(columns={'github repository': 'repo_name'})
    dummies = df['license'].apply(lambda x: x if isinstance(x, list) else []).str.join('|').str.get_dummies(sep='|')
    dummies = dummies.rename(columns=lambda x: x.replace(' ', '_'))
    return pd.concat([df, dummies], axis=1)
    
df_license = ProcessLicenseData(indir_license)
df_license_indicators = AddIndicators(df_license)

**Truck Factor**
- Obtained truck factor per time-period
- Obtained "creation date" based off earliest commit date (<span style="color:blue">There are some nonsensical commit dates so I should filter for only dates after git/svn was created</span>)
- Merged each contributor to their set of emails and identified whether they had a [company domain](https://github.com/liaochris/undergrad_thesis/blob/main/data/inputs/company_domain_match_list.yaml), a .edu email or another domain. Then, I found the proportion of contributors with educational, or corporate email domains in each project and time period <span style="color:blue"> company + industry list </span>


In [ ]:
def CleanTruckFactor(indir_truckfactor, time_period):
    df_truckfactor = pd.read_csv(indir_truckfactor / 'truckfactor.csv')
    df = df_truckfactor.dropna().copy()
    df['repo_name'] = (
        df['repo_name']
        .str.replace('drive/output/scrape/get_weekly_truck_factor/truckfactor_', '')
        .str.replace('.csv', '')
        .str.replace('_', '/', 1)
    )
    df = df.rename(columns={'date': 'created_at'})
    df = ImputeTimePeriod(df, time_period)
    
    df_truckfactor_period = (
        df
        .sort_values('created_at')
        .groupby(['time_period', 'repo_name'], as_index=False)
        .tail(1)
        .sort_values(['repo_name', 'time_period'])
    )
    df_truckfactor_period['authors'] = df_truckfactor_period['authors'].str.split(r'\s*\|\s*')
    df_truckfactor_period = df_truckfactor_period[['repo_name', 'time_period', 'created_at', 'week', 'year', 'truckfactor', 'authors']]

    return df_truckfactor_period


def AggregateCommitters(indir_committers):
    df_committers = pd.read_csv(indir_committers / 'committers_info.csv', index_col=0)
    df_clean = df_committers[['repo_name', 'commit_name', 'email_address', 'actor_id']].copy()
    df_clean[['commit_name', 'email_address']] = df_clean[['commit_name', 'email_address']].applymap(ast.literal_eval)
    
    df_grouped = (
        df_clean
        .explode('commit_name')
        .groupby(['commit_name', 'actor_id'])['repo_name']
        .apply(list)
        .reset_index()
    )
    
    actor_emails = (
        df_clean
        .explode('email_address')[['email_address', 'actor_id']]
        .drop_duplicates()
        .dropna()
        .groupby('actor_id')['email_address']
        .apply(list)
        .reset_index()
    )
    
    return df_grouped, actor_emails, df_clean

def MergeTruckFactorEmail(df_truckfactor_period, df_committers_grouped, actor_email_addresses):
    df = df_truckfactor_period.explode('authors')
    df = pd.merge(
        df,
        df_committers_grouped.rename(columns={'repo_name': 'repo'}),
        how='left',
        left_on='authors',
        right_on='commit_name'
    )
    
    group_cols = ['repo_name', 'time_period', 'authors']
    df['merge_count'] = df.groupby(group_cols)['truckfactor'].transform('count')
    df['repo_equal'] = (
        df['repo'].apply(lambda x: isinstance(x, list)) &
        df.apply(lambda x: x['repo_name'] in x['repo'] if isinstance(x['repo'], list) else False, axis=1)
    ).astype(int)
    df['merge_repo_match'] = df.groupby(group_cols)['repo_equal'].transform('sum')
    
    condition_na = (df['merge_count'] > 1) & ~((df['merge_repo_match'] == 1) & (df['repo_equal'] == 1))
    df_na = df[condition_na].drop(['merge_count', 'repo_equal', 'merge_repo_match', 'repo'], axis=1)
    df_na['commit_name'] = np.nan
    
    condition_valid = (df['merge_count'] == 1) | ((df['merge_count'] > 1) & (df['merge_repo_match'] == 1) & (df['repo_equal'] == 1))
    df_valid = df[condition_valid].drop(['merge_count', 'repo_equal', 'merge_repo_match', 'repo'], axis=1)
    
    df_agg = pd.concat([df_valid, df_na.drop_duplicates()], ignore_index=True)
    df_agg = df_agg.sort_values('commit_name').drop_duplicates(['repo_name', 'time_period', 'authors']).sort_values(['repo_name', 'time_period', 'authors'])
    df_agg = pd.merge(df_agg, actor_email_addresses, how='left')
    
    return df_agg

def AddCompanyAndIndustryColumns(df_truckfactor_agg):
    with open('issue/company_domain_match_list.yaml', 'r') as f:
        domains_list = yaml.load(f, Loader=yaml.FullLoader)
    
    domain_to_company = {}
    domain_to_industry = {}
    regex_mappings = []
    
    for entry in domains_list:
        company = entry.get('company')
        industry = entry.get('industry')
        for domain in entry.get('domains', []):
            domain_to_company[domain.lower()] = company
            domain_to_industry[domain.lower()] = industry

    
    df_truckfactor_agg['domain'] = df_truckfactor_agg['email_address'].apply(
        lambda x: [re.search(r'@([^@]+)$', email).group(1) for email in x  if isinstance(email, str) and re.search(r'@([^@]+)$', email)]
        if isinstance(x, list) else np.nan)
    df_truckfactor_agg['company'] = df_truckfactor_agg['domain'].apply(lambda x: [domain_to_company.get(email, np.nan) for email in x]
                                       if isinstance(x, list) else np.nan)
    df_truckfactor_agg['industry'] = df_truckfactor_agg['domain'].apply(lambda x: [domain_to_industry.get(email, np.nan) for email in x]
                                       if isinstance(x, list) else np.nan)
    
    for col in ['company','industry']:
        df_truckfactor_agg[col] = df_truckfactor_agg[col].apply(lambda x: sorted(set([email for email in x if not pd.isnull(email)])) 
                                                                if isinstance(x, list) else x)
        df_truckfactor_agg[col] = df_truckfactor_agg[col].apply(lambda x: x if isinstance(x, list) and len(x)>0 else np.nan)
        
    return df_truckfactor_agg

def AddEducationalColumns(df_truckfactor_agg):
    df_truckfactor_agg['educational'] = df_truckfactor_agg['email_address'].apply(
        lambda x: any([email.endswith(".edu") for email in x]) if isinstance(x, list) else False)
    return df_truckfactor_agg

def UniqueNonNull(s):
    return s.dropna().unique()

indir_truckfactor = Path('drive/output/scrape/get_weekly_truck_factor')
df_truckfactor_period = CleanTruckFactor(indir_truckfactor, time_period)

indir_committers = Path('drive/output/scrape/link_committers_profile')
df_committers_grouped, actor_email_addresses, df_committers_clean = AggregateCommitters(indir_committers)

df_truckfactor_agg = MergeTruckFactorEmail(df_truckfactor_period, df_committers_grouped, actor_email_addresses)
df_truckfactor_agg = AddCompanyAndIndustryColumns(df_truckfactor_agg)
df_truckfactor_agg = AddEducationalColumns(df_truckfactor_agg)
df_truckfactor_repo_summary = df_truckfactor_agg.assign(company_bool = 1-df_truckfactor_agg['company'].isna()).groupby(['repo_name','time_period','truckfactor']).agg(
    corporate_pct = ('company_bool','mean'),
    educational_pct = ('educational','mean')).reset_index()

# filter for after git was created? or after svn was created?
df_creation_date = df_truckfactor_period[['repo_name','created_at']].assign(
    created_year = df_truckfactor_period['created_at'].dt.year)\
    .query('created_year>1991')\
    .sort_values('created_at').drop_duplicates('repo_name')

**Organizational size**: \# of people making contributions, defined as opening/closing an issue, commenting on an issue, PR, reviewing a PR, merging a PR, or committing code

In [20]:
df_contributor_count = df_contributor_panel.groupby(['repo_name','time_period'])['actor_id'].count().reset_index().rename(
    {'actor_id':'contributors'}, axis = 1)

### Combine Covariates

In [251]:
def MakeBalanced(df_repo_panel):
    df_repo_panel['first_period'] = df_repo_panel.groupby('repo_name')['time_period'].transform('min')
    df_repo_panel['final_period'] = df_repo_panel.groupby('repo_name')['time_period'].transform('max')
    time_periods = df_repo_panel['time_period'].unique().tolist()
    df_balanced = df_repo_panel[['repo_name']].drop_duplicates()
    df_balanced['time_period'] = [time_periods for i in range(df_balanced.shape[0])]
    df_balanced = df_balanced.explode('time_period')
    df_repo_panel_full = pd.merge(df_balanced, df_repo_panel, how = 'left')
    df_repo_panel_full[['first_period','final_period']] = df_repo_panel_full.groupby(['repo_name'])[['first_period','final_period']].ffill()
    df_repo_panel_full = df_repo_panel_full.query('time_period >= first_period & time_period <= final_period')

    return df_repo_panel_full.drop(['first_period','final_period'], axis = 1).sort_values(['repo_name','time_period'])

In [252]:
df_repo_characteristics = reduce(lambda left, right: left.merge(right, how='outer'),
    [
        df_stars,
        df_fork,
        df_license_indicators.assign(
            license=df_license_indicators['license'].apply(lambda x: "|".join(x) if isinstance(x, list) and x else np.nan)
        ),
        df_creation_date,
        df_truckfactor_repo_summary,
        df_contributor_count
    ]
)

df_repo_characteristics = MakeBalanced(df_repo_characteristics)
df_repo_characteristics.columns = df_repo_characteristics.columns.str.lower()

na_zero_cols = ['stars_accumulated', 'forks_accumulated', 'contributors']
df_repo_characteristics[na_zero_cols] = df_repo_characteristics[na_zero_cols].fillna(0)

fill_cols = [
    'license', 'apache_software_license', 'bsd_license',
    'gnu_general_public_license', 'mit_license', 'other_license',
    'created_at', 'created_year'
]
df_repo_characteristics[fill_cols] = df_repo_characteristics.groupby('repo_name')[fill_cols].transform(lambda x: x.ffill().bfill())

truck_cols = ['truckfactor', 'corporate_pct', 'educational_pct']
df_repo_characteristics[truck_cols] = df_repo_characteristics.groupby('repo_name')[truck_cols].transform('ffill')

## Individual Characteristics

**Email domain**: Whether in a project-contributor-time period, an individual made a commit with an educational or corporate email domain

In [194]:
df_contributor_email_category = df_truckfactor_agg.assign(company_bool = ~df_truckfactor_agg['company'].isna())\
    .groupby(['repo_name','actor_id','time_period'])\
    .agg(contributor_email_educational = ('educational','max'), 
         contributor_email_corporate = ('company_bool','max')).reset_index()

1) Max Rank for each individual in each period as a continuous rank variable
2) Rank indicator
3) % for each rank that they handled

df_contributor_panel

In [199]:
def ContributorRankCharacteristics(df_contributor_panel):
    val_cols = [col for col in df_contributor_panel.columns if 'pct' in col]
    df = df_contributor_panel.drop(columns=val_cols).rename(columns={'issue_number': 'issues_opened', 'pr': 'prs_opened'}).reset_index(drop=True)
    
    df['problem_identification_sum'] = df['issues_opened']
    df['problem_identification'] = df['problem_identification_sum'] > 0
    
    df['problem_discussion_sum'] = df[['comments', 'own_pr_comments', 'helping_pr_comments', 'pr_reviews', 'pr_review_comments']].sum(axis=1)
    df['problem_discussion'] = df['problem_discussion_sum'] > 0
    
    df['coding_sum'] = df[['prs_opened', 'commits']].sum(axis=1)
    df['coding'] = df['coding_sum'] > 0
    
    df['problem_approval_sum'] = df[['prs_merged', 'issues_closed']].sum(axis=1)
    df['problem_approval'] = df['problem_approval_sum'] > 0
    
    df['max_rank'] = np.select(
        [
            df['problem_approval'],
            df['coding'],
            df['problem_discussion']
        ],
        [4, 3, 2],
        default=1
    )
    agg_cols = ['problem_identification', 'problem_discussion', 'coding', 'problem_approval']
    grouped_sums = df.groupby(['repo_name', 'time_period'])[agg_cols].transform('sum')
    for col in agg_cols:
        df[f'{col}_share'] = df[col] / grouped_sums[col]
    rank_activity_cols = [
        'issues_opened', 'problem_identification_sum', 'comments', 'own_pr_comments',
        'helping_pr_comments', 'pr_reviews', 'pr_review_comments', 'problem_discussion_sum',
        'prs_opened', 'commits', 'coding_sum', 'prs_merged', 'issues_closed',
        'problem_approval_sum'
    ]
    final_cols = ['actor_id', 'repo_name', 'time_period', 'max_rank'] + agg_cols + [f'{c}_share' for c in agg_cols] + rank_activity_cols
    return df[final_cols]

df_contributor_panel_rank = ContributorRankCharacteristics(df_contributor_panel)

In [202]:
agg_cols = ['problem_identification', 'problem_discussion', 'coding', 'problem_approval']
df_contributor_panel_rank[['actor_id','repo_name','time_period','max_rank'] + [f'{c}_share' for c in agg_cols]]

,actor_id,repo_name,time_period,max_rank,problem_identification_share,problem_discussion_share,coding_share,problem_approval_share
0,376272.0,1e0ng/simhash,2020-07-01,3,0.500000,0.500000,0.5,0.000000
1,1264873.0,1e0ng/simhash,2020-07-01,4,0.500000,0.500000,0.5,1.000000
2,1264873.0,1e0ng/simhash,2021-01-01,2,0.333333,0.333333,NaN,0.000000
3,1264873.0,1e0ng/simhash,2021-07-01,4,0.142857,0.142857,1.0,0.333333
4,1264873.0,1e0ng/simhash,2022-01-01,4,0.166667,0.166667,0.5,1.000000
...,...,...,...,...,...,...,...,...
1554030,443794.0,zzzsochi/trans,2016-01-01,3,NaN,NaN,1.0,NaN
1554031,443794.0,zzzsochi/trans,2016-07-01,4,0.333333,0.333333,0.5,1.000000
1554032,2354108.0,zzzsochi/trans,2016-07-01,2,0.333333,0.333333,0.0,0.000000
1554033,4735252.0,zzzsochi/trans,2015-07-01,4,0.000000,0.000000,0.0,0.500000


Part of truck factor, indicator
- drive/output/scrape/get_weekly_truck_factor


In [208]:
df_truckfactor_dates = df_truckfactor_agg[['repo_name','time_period','actor_id']].assign(
    truckfactor_member = 1)

In [274]:
#df_contributor_panel
#df_issue.query('repo_name == "1e0ng/simhash" & actor_id == 376272.0')

,type,created_at,repo_id,repo_name,actor_id,actor_login,pr_title,pr_number,pr_body,pr_action,pr_merged_by_id,pr_merged_by_type,pr_label,pr_review_action,pr_review_id,pr_review_body,pr_review_state,pr_review_comment_body,date,year,period,time_period
15463,PullRequestEvent,2020-10-16 15:25:56+00:00,12833146.0,1e0ng/simhash,1264873.0,1e0ng,None,50.0,None,closed,1264873.0,User,[],None,NaN,None,None,None,2020-10,2020,1,2020-07-01
40847,PullRequestEvent,2020-10-06 21:09:52+00:00,12833146.0,1e0ng/simhash,376272.0,jcushman,None,50.0,None,opened,NaN,None,[],None,NaN,None,None,None,2020-10,2020,1,2020-07-01
12040,PullRequestReviewEvent,2020-10-09 13:09:15+00:00,12833146.0,1e0ng/simhash,1264873.0,1e0ng,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,created,505652288.0,None,commented,None,2020-10,2020,1,2020-07-01
12559,PullRequestReviewEvent,2020-10-09 19:59:56+00:00,12833146.0,1e0ng/simhash,376272.0,jcushman,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,created,505955117.0,None,commented,None,2020-10,2020,1,2020-07-01
17133,PullRequestReviewEvent,2020-10-16 15:25:42+00:00,12833146.0,1e0ng/simhash,1264873.0,1e0ng,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,created,510575371.0,None,approved,None,2020-10,2020,1,2020-07-01
24597,PullRequestReviewEvent,2020-10-09 20:06:32+00:00,12833146.0,1e0ng/simhash,376272.0,jcushman,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,created,505958608.0,None,commented,None,2020-10,2020,1,2020-07-01
3385,PullRequestReviewCommentEvent,2020-10-09 13:05:14+00:00,12833146.0,1e0ng/simhash,1264873.0,1e0ng,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,None,NaN,None,None,👍,2020-10,2020,1,2020-07-01
6489,PullRequestReviewCommentEvent,2020-10-09 19:59:55+00:00,12833146.0,1e0ng/simhash,376272.0,jcushman,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,None,NaN,None,None,I selected both of those values based on where...,2020-10,2020,1,2020-07-01
7889,PullRequestReviewCommentEvent,2020-10-09 13:04:31+00:00,12833146.0,1e0ng/simhash,1264873.0,1e0ng,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,None,NaN,None,None,Could you elaborate on how 50 is calculated? A...,2020-10,2020,1,2020-07-01
8885,PullRequestReviewCommentEvent,2020-10-09 13:09:01+00:00,12833146.0,1e0ng/simhash,1264873.0,1e0ng,Use numpy to calculate simhash,50.0,"Per #49, here is code to use numpy for calcula...",None,NaN,None,None,None,NaN,None,None,👍,2020-10,2020,1,2020-07-01


Git blame (%)
- <p style = "color:blue"> can't do yet</p>
- May be helpful: https://github.com/HelgeCPH/truckfactor/blob/main/truckfactor/compute.py

Grouped at the period level but can also consider aggregating across periods

For each (2), (3), (4) that they were involved in
1) what \% of the work did they do
2) what was their HHI
- Can weight by average or comment-weighted average
3) what was the \% of work they're involved in that had other people involved

**notes**
- Use drive/output/derived/data_export/df_issue.parquet, drive/output/derived/data_export/df_pr_commits.parquet, drive/output/derived/data_export/df_pr.parquet, drive/output/derived/data_export/df_push_commits.parquet
- (1) can't have HHI since only one person can open each issue, also (1) has share calculated already
- (3) can just be code commits on merged PRs
- (4) also can't have HHI  since only one person can close/merge each issue/PR, also (4) has share calculated already

note that the same discussion can be in multiple time periods

In [27]:
df_issue = pd.read_parquet('drive/output/derived/data_export/df_issue.parquet')
df_issue = ImputeTimePeriod(df_issue, time_period)

df_pr = pd.read_parquet('drive/output/derived/data_export/df_pr.parquet')
df_pr = ImputeTimePeriod(df_pr, time_period)

df_pr_commits = pd.read_parquet('drive/output/derived/data_export/df_pr_commits.parquet')
df_pr_commits['created_at'] = pd.to_datetime(df_pr_commits['commit time'], unit = 's')
df_pr_commits = df_pr_commits.query('~created_at.isna()')
df_pr_commits = ImputeTimePeriod(df_pr_commits, time_period)

In [28]:
df_issue_comments = df_issue.query('type == "IssueCommentEvent"').sort_values('created_at').drop_duplicates('issue_comment_id')
df_pr_rc = df_pr.query('type == "PullRequestReviewCommentEvent"')
df_pr_rc['issue_number'] =  df_pr_rc['pr_number']
df_pr_rc = df_pr_rc.sort_values('created_at').drop_duplicates('pr_review_comment_body')

df_comments_actor = pd.concat([df_pr_rc[['repo_name','actor_id','time_period','issue_number','created_at']],
                         df_issue_comments[['repo_name','actor_id','time_period','issue_number','created_at']]])\
    .groupby(['repo_name','actor_id','time_period','issue_number'])['created_at'].count()\
    .reset_index().rename({'created_at':'comments'}, axis = 1)

In [29]:
def AggregateHHICount(df, id_col, count_col):
    indiv_total_col = f'indiv_total_{count_col}'
    discussions_col = f'{count_col}_discussions_involved'
    total_col = f'total_{count_col}'
    share_col = f'{count_col}_share'
    hhi_col = f'{count_col}_hhi'
    indiv_col = f'{count_col}_indiv'
    wm = lambda x: np.average(x, weights=df.loc[x.index, count_col])

    df[discussions_col] = df.groupby(['repo_name','time_period','actor_id'])[count_col].transform('count')
    df[indiv_total_col] = df.groupby(['repo_name','time_period','actor_id'])[count_col].transform('sum')
    df[total_col] = df.groupby(['repo_name','time_period',id_col])[count_col].transform('sum')
    df[share_col] = df[count_col]/df[total_col]
    df[hhi_col] = df.assign(share_sq = lambda x: x[share_col]**2)\
        .groupby(['repo_name','time_period',id_col])['share_sq'].transform('sum')
    df[indiv_col] = (df[share_col]<1).astype(int)
    df[f'{share_col}_wt'] = df[share_col] * df[count_col]/df[indiv_total_col]
    df[f'{hhi_col}_wt'] = df[hhi_col] * df[count_col]/df[indiv_total_col]
    
    df_grouped = df.groupby(['repo_name','actor_id','time_period',indiv_total_col,discussions_col]).agg(
        **{f'{share_col}_avg': (share_col, 'mean'),
           f'{share_col}_avg_wt': (f'{share_col}_wt', 'sum'),
           f'{hhi_col}_avg': (hhi_col, 'mean'),
           f'{hhi_col}_avg_wt': (f'{hhi_col}_wt', 'sum'),
           f'pct_cooperation_{count_col}': (indiv_col, 'mean')}).reset_index()
    
    return df_grouped

In [30]:
df_comments_grouped = AggregateHHICount(df_comments_actor, 'issue_number', 'comments')

In [31]:
df_committers_ids = df_committers_clean.explode('commit_name').explode('email_address')[['commit_name','email_address','actor_id']].drop_duplicates()
df_pr_commits_id = pd.merge(df_pr_commits, df_committers_ids, how = 'left', left_on = ['commit author name','commit author email'], right_on = ['commit_name','email_address'])
df_pr_commits_id['commits'] = 1
df_pr_commits_id['commits_lt_100_fc'] = (df_pr_commits_id['commit files changed count']<100).astype(int)

#### NOT GOING TO WEIGHT BY ADDITIONS, DELETIONS FOR NOW BUT GOOD TO HAVE
df_pr_commits_actor = df_pr_commits_id.query('~actor_id.isna()').groupby(['repo_name','time_period','actor_id','pr_number']).agg(
    commits_count = ('commits','sum'),
    commits_lt_100_fc_count = ('commits_lt_100_fc','sum'),
    commits_add_sum = ('commit additions','sum'),
    commits_del_sum = ('commit deletions','sum'),
    commits_change_sum = ('commit changes total','sum'),
    commits_files_changed_count = ('commit files changed count','sum')).reset_index()

In [32]:
# identify for each PR unique commits
# separate analysis for commits with <100 files
# also do stuff with file changes

df_pr_commits_grouped = AggregateHHICount(df_pr_commits_actor, 'pr_number', 'commits_count')
df_pr_commits_grouped = df_pr_commits_grouped.rename({'commits_count_discussions_involved':'pr_discussions_involved'}, axis = 1)

df_pr_commits_lt_100_fc_grouped = AggregateHHICount(df_pr_commits_actor, 'pr_number', 'commits_lt_100_fc_count')
df_pr_commits_lt_100_fc_grouped = df_pr_commits_lt_100_fc_grouped.rename({'commits_lt_100_fc_count_discussions_involved':'lt_100_fc_pr_discussions_involved'}, axis = 1)

## Organizational Structure

**Layer Count**
- For each layer, %  of all individuals involved in a layer, # of people involved
- Also number of layers where there's someone in that layer uniquely and not in any other layer 

df_contributor_rank - I think I can aggregate this


In [33]:
import itertools

LAYERS = agg_cols

def find_all_min_covers_for_group(grp):
    """
    Given the subset of the DataFrame for one (repo_name, time_period),
    return ALL minimal subsets of LAYERS that cover every actor in this group.

    Cover criterion:
      For each row (actor), at least one column in the chosen subset is True.
    """
    # We'll just slice the boolean columns
    subset_matrix = grp[LAYERS]

    # We'll track all minimal subsets found
    # If we never find any, it returns an empty list (though that likely won't happen
    # if there's at least one True in each group).
    min_solutions = []

    # 1) We try subsets of size 1, then size 2, etc., up to size 4
    # 2) Once we find ANY solutions for a given size r, we stop (those are minimal).
    for r in range(1, len(LAYERS) + 1):
        possible_combos = itertools.combinations(LAYERS, r)
        found_any = False

        for combo in possible_combos:
            # If every row is covered by 'combo', we keep it
            # "covered" means row-wise .any(axis=1) is True for all rows
            if subset_matrix[list(combo)].any(axis=1).all():
                min_solutions.append(combo)
                found_any = True

        if found_any:
            # We found at least one solution of size r, so there's no need
            # to look at larger subsets.
            break

    return pd.Series({'all_min_layers': min_solutions})

df_panel_min_layers = df_contributor_panel_rank\
    .groupby(['repo_name', 'time_period'], as_index=False)\
    .parallel_apply(find_all_min_covers_for_group)
df_panel_min_layers['min_layer_size'] = df_panel_min_layers['all_min_layers'].apply(lambda x: len(x[0]))

In [34]:
df_panel_rank_count = df_contributor_panel_rank.groupby(['repo_name','time_period'])[agg_cols].sum()
df_panel_rank_count.columns = [f"{col}_layer_count" for col in df_panel_rank_count.columns]
df_panel_rank_pct = df_contributor_panel_rank.groupby(['repo_name','time_period'])[agg_cols].mean()
df_panel_rank_pct.columns = [f"{col}_layer_pct" for col in df_panel_rank_count.columns]
df_panel_rank_count_pct = df_panel_rank_pct.join(df_panel_rank_count, how = 'outer').reset_index()
df_panel_rank = df_panel_rank_pct.join(df_panel_rank_count, how = 'outer').reset_index()

df_panel_layers = (df_panel_rank_count>0).sum(axis = 1).reset_index().rename({0:'layer_count'}, axis = 1)
df_panel_layers = pd.merge(df_panel_min_layers, df_panel_layers)

#df_panel_rank df_panel_layers

**overlap**
- For each overlap combination (of two), % overlap between the areas
- Degree of overlap - % of work done in lower layer by someone whose in a specific higher layer
- Degree of overlap - % of work done in lower layer by someone whose also in a higher layer (sum of the above, can also try)

always use lower layer as nominator

df_contributor_panel_rank can also be used

In [35]:
df_repo_panel_overlap = df_contributor_panel_rank[['repo_name', 'time_period']].drop_duplicates().reset_index(drop=True)

for lower_layer, higher_layer in itertools.combinations(LAYERS, 2):
    
    overlap = (
        df_contributor_panel_rank
        .query(f'({lower_layer} == True)')
        .groupby(['repo_name', 'time_period'])[higher_layer]
        .mean()
        .reset_index()
        .rename(columns={higher_layer: f"{lower_layer}_{higher_layer}_overlap"})
    )
    df_repo_panel_overlap = pd.merge(df_repo_panel_overlap, overlap, on=['repo_name', 'time_period'], how='left')
    
    lower_higher_numerator = (
        df_contributor_panel_rank
        .query(f'({lower_layer} == True) & ({higher_layer} == True)')
        .groupby(['repo_name', 'time_period'])[f"{lower_layer}_sum"]
        .sum()
        .reset_index()
        .rename(columns={f"{lower_layer}_sum": f"{lower_layer}_{higher_layer}_work_done_numerator"})
    )
    lower_higher_denominator = (
        df_contributor_panel_rank
        .query(f'({lower_layer} == True)')
        .groupby(['repo_name', 'time_period'])[f"{lower_layer}_sum"]
        .sum()
        .reset_index()
        .rename(columns={f"{lower_layer}_sum": f"{lower_layer}_{higher_layer}_work_done_denominator"})
    )
    
    work_done = pd.merge(
        lower_higher_numerator, 
        lower_higher_denominator, 
        on=['repo_name', 'time_period'], 
        how='right'  # Ensure all groups with lower_layer=True are included
    )
    
    work_done[f"{lower_layer}_{higher_layer}_work_done"] = work_done.apply(
        lambda row: (
            row[f"{lower_layer}_{higher_layer}_work_done_numerator"] / 
            row[f"{lower_layer}_{higher_layer}_work_done_denominator"]
        ) if row[f"{lower_layer}_{higher_layer}_work_done_denominator"] > 0 else 0,
        axis=1
    )
    
    # e) Select Relevant Columns and Fill NaN with 0
    work_done = work_done[['repo_name', 'time_period', f"{lower_layer}_{higher_layer}_work_done"]]
    
    # Merge the work_done data into the summary DataFrame
    df_repo_panel_overlap = pd.merge(df_repo_panel_overlap, work_done, on=['repo_name', 'time_period'], how='left')

df_repo_panel_overlap

,repo_name,time_period,problem_identification_problem_discussion_overlap,problem_identification_problem_discussion_work_done,problem_identification_code_writing_overlap,problem_identification_code_writing_work_done,problem_identification_problem_approval_overlap,problem_identification_problem_approval_work_done,problem_discussion_code_writing_overlap,problem_discussion_code_writing_work_done,problem_discussion_problem_approval_overlap,problem_discussion_problem_approval_work_done,code_writing_problem_approval_overlap,code_writing_problem_approval_work_done
0,1e0ng/simhash,2020-07-01,1.0,1.0,1.000000,1.000000,0.500000,0.800000,1.000000,1.000000,0.500000,0.619048,0.5,0.571429
1,1e0ng/simhash,2021-01-01,1.0,1.0,0.000000,NaN,0.666667,0.500000,0.000000,NaN,0.666667,0.500000,NaN,NaN
2,1e0ng/simhash,2021-07-01,1.0,1.0,0.142857,0.631579,0.428571,0.789474,0.142857,0.583333,0.428571,0.722222,1.0,1.000000
3,1e0ng/simhash,2022-01-01,1.0,1.0,0.166667,0.500000,0.166667,0.500000,0.166667,0.500000,0.166667,0.500000,0.5,0.571429
4,4Catalyzer/flask-resty-tenants,2016-01-01,1.0,1.0,0.333333,0.428571,0.333333,0.428571,0.333333,0.450000,0.333333,0.450000,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87905,zyga/guacamole,2015-01-01,1.0,1.0,1.000000,1.000000,0.000000,NaN,1.000000,1.000000,0.000000,NaN,0.0,NaN
87906,zyga/guacamole,2018-01-01,1.0,1.0,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN
87907,zzzsochi/trans,2015-07-01,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000
87908,zzzsochi/trans,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


**work split**
- HHI for each layer

df_contributor_panel

- git blame, for each file, find the HHI
  - average using all files
  - average using LOC


<p style = "color:blue"> can't do yet</p>
- May be helpful: https://github.com/HelgeCPH/truckfactor/blob/main/truckfactor/compute.py

In [36]:
df_repo_hhi = df_contributor_panel_rank[['repo_name', 'time_period']].drop_duplicates().reset_index(drop=True)

# Iterate through each layer to calculate HHI
for layer in LAYERS:
    share_col = f"{layer}_share"
    hhi_col = f"{layer}_HHI"
    hhi = (
        df_contributor_panel_rank
        .query(f"{layer} == True")
        .assign(share_sq = df_contributor_panel_rank[share_col]**2)
        .groupby(['repo_name', 'time_period'])['share_sq']
        .sum()
        .reset_index()
        .rename(columns={'share_sq': hhi_col})
    )
    
    df_repo_hhi = pd.merge(df_repo_hhi, hhi, on=['repo_name', 'time_period'], how='left')
# df_repo_hhi

**cooperation**
- calculate HHI for each discussion thread (2), PR linked commits (3) and PR review (4)
  - can use average across discussions or average weighted by qty of discussion
- \# of distinct individuals per discussion
  - can use average across discussions or average weighted by qty of discussion
- % of each with more than one distinct individual

Use drive/output/derived/data_export/df_issue.parquet, drive/output/derived/data_export/df_pr_commits.parquet, drive/output/derived/data_export/df_pr.parquet, drive/output/derived/data_export/df_push_commits.parquet

In [37]:
import pandas as pd
import itertools

def calculate_hhi_cooperation(df, entity, metric):
    metric_share = f"{metric}_share"                # e.g., 'comments_share'
    entity_number = f"{entity}_number"              # e.g., 'issue_number'
    metric_col = metric                             # e.g., 'comments'
    total_metric = f"total_{metric}"                # e.g., 'total_comments'
    repo_metric = f"repo_{metric}"                  # e.g., 'repo_comments'
    
    hhi_col = f"{entity}_hhi_{metric}"              # e.g., 'issue_hhi_comments'
    cooperation_col = f"{metric}_cooperation"       # e.g., 'comments_cooperation'
    wt_work_done_col = f"{entity}_{metric}_wt"         # e.g., 'issue_comments_wt'
    
    df[hhi_col] = df.assign(share_sq = df[metric_share] ** 2)\
        .groupby(['repo_name', 'time_period', entity_number])['share_sq'].transform('sum')
    
    df[cooperation_col] = df[hhi_col] != 1
    df[repo_metric] = df.groupby(['repo_name', 'time_period'])[metric_col].transform('sum')
    
    df[wt_work_done_col] = df[hhi_col] * df[total_metric] / df[repo_metric]
    
    df_unique = df.drop_duplicates(['repo_name', 'time_period', entity_number])
    
    df_cooperation = df_unique.groupby(['repo_name', 'time_period']).agg(
        hhi_metric = (hhi_col, 'mean'),
        hhi_metric_wt = (wt_work_done_col, 'sum'),
        metric_cooperation_pct = (cooperation_col, 'mean')
    ).reset_index()
    
    df_cooperation = df_cooperation.rename(columns={
        'hhi_metric': f"hhi_{metric}",
        'hhi_metric_wt': f"hhi_{metric}_wt",
        'metric_cooperation_pct': f"{metric}_cooperation_pct"
    })
    
    
    return df_cooperation


In [38]:
df_comments_cooperation = calculate_hhi_cooperation(df_comments_actor, 'issue','comments')
df_commits_cooperation = calculate_hhi_cooperation(df_pr_commits_actor, 'pr','commits_count')

**communication**
- Given a contributor's rank, what % of discussion occurs with people whose rank is (1), (2), (3), (4)
  - get 1/x weight if they are in x ranks (for the % of discussion)
  - Average weighted by contributions of person, or by person
- Can aggregate for each contributor into a weighted average of the average rank they communicate with 

In [39]:
df_comments_actor_rank = pd.merge(df_comments_actor, df_contributor_panel_rank[['actor_id','repo_name','time_period','max_rank'] + agg_cols].drop_duplicates(), how = 'left')

In [40]:
def RankParticipationFlags(df, entity='issue'):
    entity_number = f"{entity}_number"

    ranks = [1, 2, 3, 4]

    rank_cols = [f'rank_{rank}' for rank in ranks]
    for rank in ranks:
        df[f'rank_{rank}'] = (df['max_rank'] == rank).astype(int)

    group_cols = ['repo_name', 'time_period', entity_number]
    rank_presence = df.groupby(group_cols)[rank_cols].max().reset_index()

    participation_cols = [f'participated_rank_{rank}' for rank in ranks]
    rename_dict = {f'rank_{rank}': f'participated_rank_{rank}' for rank in ranks}
    rank_presence = rank_presence.rename(columns=rename_dict)

    df = df.merge(rank_presence, on=group_cols, how='left')
    df = df.drop(columns=rank_cols)

    return df

def calculate_rank_metrics(df, entity_number='issue_number', metric='comments', ranks=[1,2,3,4],
                           weight_column='comments_discussions_involved'):

    df = RankParticipationFlags(df)
    # weighted by number of ppl in discussion 
    df_ppl_wt = df.groupby(['repo_name','time_period','max_rank']).agg(
        participated_rank_1 = ('participated_rank_1','mean'),
        participated_rank_2 = ('participated_rank_2','mean'),
        participated_rank_3 = ('participated_rank_3','mean'),
        participated_rank_4 = ('participated_rank_4','mean'),
        issue_rank_count = ('actor_id','count')
    )

    # issue by issue average
    df_wt_by_issue = df.drop_duplicates(['repo_name','time_period',entity_number,'max_rank']).groupby(['repo_name','time_period','max_rank']).agg(
        participated_rank_1_wt_by_issue = ('participated_rank_1','mean'),
        participated_rank_2_wt_by_issue = ('participated_rank_2','mean'),
        participated_rank_3_wt_by_issue = ('participated_rank_3','mean'),
        participated_rank_4_wt_by_issue = ('participated_rank_4','mean')
    )

    ## ADD OTHER WEIGHTS I WANT TO DO
    df_rank_count = df.drop_duplicates(['repo_name','time_period','max_rank','actor_id']).groupby(
        ['repo_name','time_period','max_rank'])['actor_id'].count().rename('rank_contributor_count')
    
    df_rank_summary = df_ppl_wt.join(df_wt_by_issue).join(df_rank_count)
    
    return df_rank_summary

In [41]:
df_comments_rank_summary = calculate_rank_metrics(df_comments_actor_rank)

In [ ]:
def CalculateSize(df_contributor_panel):
    df_repo_size = df_contributor_panel.groupby(['repo_name','time_period']).agg({
        'actor_id':'count','problem_identification': 'sum','problem_discussion':'sum','problem_solving':'sum',
        'solution_incorporation':'sum'}).rename({
        'actor_id':"contributor_count","problem_identification":"problem_identifier_count","problem_discussion":"problem_discusser_count",
        "problem_solving":"problem_solver_count","solution_incorporation":"solution_incorporator_count"}, axis = 1).reset_index()
    return df_repo_size



In [ ]:
df_repo_size = CalculateSize(df_contributor_panel)


In [ ]:
df_repo_panel = pd.merge(df_repo_size, df_repo_hhi, how = 'outer').merge(df_repo_overlap, how = 'outer')
df_repo_panel['first_period'] = df_repo_panel.groupby('repo_name')['time_period'].transform('min')
df_repo_panel['final_period'] = df_repo_panel.groupby('repo_name')['time_period'].transform('max')

In [ ]:
time_periods = df_repo_panel['time_period'].unique().tolist()
df_balanced = df_repo_panel[['repo_name']].drop_duplicates()
df_balanced['time_period'] = [time_periods for i in range(df_balanced.shape[0])]
df_balanced = df_balanced.explode('time_period')
df_repo_panel_full = pd.merge(df_balanced, df_repo_panel, how = 'left')
df_repo_panel_full[['first_period','final_period']] = df_repo_panel_full.groupby(['repo_name'])[['first_period','final_period']].ffill()
df_repo_panel_full = df_repo_panel_full.query('time_period >= first_period')
df_repo_panel_full[[col for col in df_repo_panel_full.columns if 'hhi' not in col]] = df_repo_panel_full[[col for col in df_repo_panel_full.columns if 'hhi' not in col]].fillna(0)

In [ ]:
df_repo_overlap = CalculateOverlap(df_contributor_panel)
#df_repo_panel_full
#df_contributor_selected_panel